# Amazon Bedrock boto3 Setup

> **Pre-requisites**:
> - [Python 3.9](https://www.python.org/downloads/)

> **Setup environment**:
> - `python3.9 -m venv .venv`
> - `source .venv/bin/activate`
> - `pip install -r requirements.txt`
> ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

- We will use AWS SDK directly to interact with AWS services.

### AWS Services

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

### agents

In [ ]:
%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    google-search-results

### entities

In [ ]:
%pip install --quiet beautifulsoup4


### image

In [ ]:
%pip install --quiet "pillow>=9.5,<10"


### gaurdrails

%%brew update
%%brew install gcc


#### Uninstall unwanted packages

In [ ]:
!python -m pip uninstall langchain -y
%pip uninstall langchain_experimental -y
%pip uninstall langchain-0.0.352 -y
%pip uninstall langchain_experimental -y

### nemoguardrails

In [ ]:
%pip install -qU --no-cache-dir nemoguardrails==0.5.0


%pip install -qU "faiss-cpu>=1.7,<2" \
                      "pypdf>=3.8,<4" \
                      "ipywidgets>=7,<8"

In [ ]:
%pip install -qU "langchain>=0.0.251" 
%pip install -qU "langchain_experimental>=0.0.30"

### Restart kernel

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup AWS Credentials

1.  Create a new IAM user with `local code` access to Amazon Bedrock resources. 
    *AmazonBedrockFullAccess* policy will provide all the required permissions to the user.
    *AmazonS3FullAccess* policy will provide access to S3 bucket.

    [Create IAM user](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console)

    ![IAM Image](https://raw.githubusercontent.com/Dinuda/BedrockBoto3Setup/main/images/confuser.png?raw=true)

2. Create a new AWS profile
    ```bash
    aws configure --profile bedrockuser

    AWS Access Key ID [None]: <access_key_id>
    AWS Secret Access Key [None]: <secret_access_key>
    Default region name [None]: <region>
    Default output format [None]: text
    ```

3. Create new AWS service role for Bedrock
   
    [Create service role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_for-service.html#roles-creatingrole-service-console)
    
    ![IAM Image](https://raw.githubusercontent.com/Dinuda/BedrockBoto3Setup/main/images/servrole.png?raw=true)
   
    -- Trust Policy
    ```
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "AWS": "arn:aws:iam::<userid>:role/service-role/AgentFineTuningServiceRole",
                    "Service": [
                        "sagemaker.amazonaws.com",
                        "bedrock.amazonaws.com",
                        "events.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    ```

4. Copy the `Role ARN`

### Create client

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))


os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "bedrockuser"
os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

boto3_bedrock = boto3.client('bedrock')


#### Validate the connection




In [ ]:
boto3_bedrock.list_foundation_models()


### Invoke the API

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime')


#### Anthropic Claude

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""


In [ ]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error
